In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
import plotnine as p9

In [46]:
data = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv').head(30000).sort_values(['state'])
data.head()

/var/folders/0x/j2sfsgp167vf2w83dyjt6_nm0000gn/T/ipykernel_11304/1193472143.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
15184,021313,2022/10/07,AK,021313,SOUTH PENINSULA HOSPITAL,4300 BARTLETT ST,HOMER,99603.0,Critical Access Hospitals,2120.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
7327,020001,2020/03/27,AK,020001,PROVIDENCE ALASKA MEDICAL CENTER,3200 PROVIDENCE DRIVE,ANCHORAGE,99508.0,Short Term,2020.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
11806,021313,2022/02/11,AK,021313,SOUTH PENINSULA HOSPITAL,4300 BARTLETT ST,HOMER,99603.0,Critical Access Hospitals,2120.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
18367,020008,2022/07/29,AK,020008,BARTLETT REGIONAL HOSPITAL,3260 HOSPITAL DR,JUNEAU,99801.0,Short Term,2110.0,...,0.0,0.0,7,0.0,-999999.0,7,-999999.0,0.0,7,0.0
15693,020008,2022/03/25,AK,020008,BARTLETT REGIONAL HOSPITAL,3260 HOSPITAL DR,JUNEAU,99801.0,Short Term,2110.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0


In [47]:
len(data)

30000

In [48]:
def ifZero(x):
    if x < 0:
        return 0
    return x

In [49]:
clean = data
for col in clean.columns:
  if clean[col].dtype != 'float64':
    clean = clean.drop(col, axis=1)
clean = clean.drop(['zip', 'fips_code'], axis=1)
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
15184,NaN,NaN,22.0,11.6,11.6,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7327,490.0,NaN,NaN,231.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11806,NaN,NaN,22.0,9.9,9.9,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18367,NaN,NaN,53.0,40.4,37.7,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,-999999.0,-999999.0,0.0,0.0
15693,NaN,NaN,53.6,36.9,33.9,-999999.0,-999999.0,-999999.0,-999999.0,-999999.0,...,0.0,NaN,-999999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6273,12.0,NaN,NaN,5.1,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3281,12.0,NaN,NaN,-999999.0,NaN,-999999.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928,12.5,NaN,NaN,-999999.0,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,42.9,NaN,NaN,4.9,NaN,-999999.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
clean = clean.dropna()
clean = clean.applymap(lambda x: ifZero(x))
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
14965,314.9,271.9,231.9,213.7,199.0,5.6,5.6,4.9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15037,1060.7,961.7,870.7,888.3,827.6,154.4,144.7,104.9,9.7,0.0,...,6.0,4.0,0.0,0.0,0.0,0.0,6.0,18.0,6.7,47.0
15042,66.1,66.1,57.3,42.7,42.7,13.0,13.0,12.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14953,177.0,177.0,147.0,93.9,93.9,31.9,31.9,31.9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13559,313.0,297.0,267.0,228.7,228.7,30.3,30.3,30.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29569,144.0,144.0,144.0,87.3,87.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29738,61.4,61.4,36.4,19.4,19.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29623,586.0,586.0,586.0,478.6,478.6,64.4,64.4,62.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29536,48.3,46.0,34.0,29.4,26.9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
output = clean['inpatient_beds_used_covid_7_day_sum'].values.reshape(-1, 1)
clean = clean.drop('inpatient_beds_used_covid_7_day_sum', axis=1)
'inpatient_beds_used_covid_7_day_sum' in clean

False

In [52]:
solve = []
maxTrainAccuracy = -1
maxTestAccuracy = -1
index = -1
for col in clean:
  input = clean[col].values.reshape(-1, 1)
  xtrain, xtest, ytrain, ytest = train_test_split(input, output, test_size=0.3, random_state=404)
  dtree = DecisionTreeClassifier(random_state=404)
  dtree.fit(xtrain, ytrain)
  y_pred = dtree.predict(xtest)
  trainAccuracy = accuracy_score(y_true = ytrain, y_pred = dtree.predict(xtrain))
  testAccuracy = accuracy_score(y_true = ytest, y_pred = y_pred)
  if testAccuracy > maxTestAccuracy:
    maxTrainAccuracy = trainAccuracy
    maxTestAccuracy = testAccuracy
    index = len(solve)
  solve.append({
    'col': col,
    'input': input,
    'output': output,
    'xtrain': xtrain,
    'xtest': xtest,
    'ytrain': ytrain,
    'ytest': ytest,
    'dtree': dtree,
    'y_pred': y_pred,
    'trainAccuracy': trainAccuracy,
    'testAccuracy': testAccuracy,
  })
  

In [59]:
print('col:',solve[index].get('col'))
print("index:",index)
print(maxTrainAccuracy)
print(maxTestAccuracy)

col: total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
index: 25
0.9172932330827067
0.8201754385964912
